In [101]:
#from pymongo import MongoClient
#import pandas as pd


In [102]:
#import tools.mongo as mongo

In [103]:
#df_companies=mongo.getCompanies(mongo.localConection())

In [104]:
#df_companies.head()


In [105]:
#sum(df_companies.deadpooled_year.value_counts().values)

929

In [106]:
#df_companies[df_companies.deadpooled_year.isnull()]

## Api google places

In [ ]:
#https://developers.google.com/places/web-service/supported_types
#types supported by google places
#next_page_token

In [3]:
import src.tools.google as google


In [4]:
a=google.getNearbyplaces(47.6918452,-122.2226413,10000,"bar")

In [ ]:
## Obtener sturbuck

In [ ]:
## dar valoracion

## Meetup api

In [91]:
#urlBase='https://api.meetup.com'

In [92]:
#urlFindEvent='/find/upcoming_events'

In [96]:
'''
params={
    'lat':'47.6918452',
    'lon':'-122.2226413'
}
'''

In [2]:
#res = requests.get('urlBase+urlFindEvent, params=params')
#res.json()